# 1. Connect to IB 

In [1]:
# In your Jupyter notebook (TEST_Daily_Analysis.ipynb)
import sys
from pathlib import Path

# Add the project root directory to Python path
ROOT_DIR = Path.cwd().parent  # Goes up one level from 'scanner' to 'stockbot4'
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

In [2]:

from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7496, clientId=13)

<IB connected to 127.0.0.1:7496 clientId=13>

# 2. Market Segment Analysis

In [3]:
import market_analyzer as ma

# Initialize analyzer
analyzer = ma.IBMarketAnalyzer(ib)

# IMPORTANT: Set up the sectors first
analyzer.set_sectors()  # This was missing in the previous code

# Now get the full analysis
market_df, sector_df = analyzer.create_analysis_report()



Running market and sector analysis first...


In [4]:
sector_df.sort_values('Overall Score', inplace=True, ascending=False)
sector_df = sector_df.round(2)
sector_df

,Sector,ETF,Return (%),Relative Strength,Momentum (ROC),Volume Trend,Outperforming SPY,Momentum Direction,Relative Strength Rank,Return Rank,Momentum Rank,Overall Score
0,Consumer_Discretionary,XLY,11.07,8.70,6.86,Increasing,Yes,Positive,1,1.0,1.0,9.00
1,Communication Services,XLC,3.98,1.61,0.56,Increasing,Yes,Positive,2,2.0,3.0,7.67
2,Technology,XLK,3.74,1.37,1.34,Increasing,Yes,Positive,3,3.0,2.0,7.33
3,Consumer_Staples,XLP,1.04,-1.33,-1.48,Increasing,No,Negative,4,4.0,4.0,6.00
4,Financials,XLF,-1.19,-3.56,-2.68,Increasing,No,Negative,5,5.0,5.0,5.00
5,Healthcare,XLV,-1.42,-3.79,-5.18,Increasing,No,Negative,6,6.0,9.0,3.00
6,Industrials,XLI,-1.81,-4.18,-4.04,Increasing,No,Negative,7,7.0,7.0,3.00
7,Real_Estate,XLRE,-2.57,-4.94,-3.72,Increasing,No,Negative,8,8.0,6.0,2.67
8,Materials,XLB,-4.08,-6.45,-6.85,Decreasing,No,Negative,9,9.0,10.0,0.67
9,Utilities,XLU,-4.66,-7.03,-4.66,Decreasing,No,Negative,10,10.0,8.0,0.67


In [5]:


# Then filter the sectors in various ways:

custom_filter_df = analyzer.filter_sectors( 
    sector_df,
    as_list_of_sectors=False,
    min_momentum=0,
    sort_by='Overall Score',
    min_score=5
)

custom_filter_list = analyzer.filter_sectors( 
    sector_df,
    as_list_of_sectors=True,
    min_momentum=0,
    sort_by='Overall Score',
    min_score=5
)

# ib.disconnect()

print("=== Market Analysis ===")
display(market_df)

print("\n=== Sector Analysis ===")
display(sector_df)


try:


    print("\n=== Custom Filter DF ===")
    display(custom_filter_df)

    print("\n=== Custom Filter Sector List ===")
    display(custom_filter_list)

except Exception as e:
    print(f"An error occurred: {str(e)}")

status_dict = {
    "sector_analysis": len(sector_df),
    "stocks_scanned": 0,
    "stocks_validated": 0,
    "stocks_scored": 0
}

# ------------------- EMAIL -------------------
from emails import email_summery, email_client

# Create the email generator and generate email
email_generator = email_summery.StockSummaryEmail()
email_html = email_generator.generate_email(
    status_dict,
    scan_settings= None,
    activity_df = None,
    sector_etf_df = sector_df.round(2),
    fundamentals_df = None,
    ta_df = None,
    daily_ta_df = None
)

email_client.send_outlook_email(
    subject="STOCKBOT: Daily Stock Summery",
    body=email_html,
    image_paths=[],
    recipients=['pary888@gmail.com'],
    is_html=True)

=== Market Analysis ===


,Metric,Description,Value,Interpretation
0,SPY Current Price,Current market price of SPY ETF,604.29,Current market price point
1,SPY 20-Day MA,20-day simple moving average - short-term trend indicator,601.69,Price above 20MA is bullish short-term
2,SPY 50-Day MA,50-day simple moving average - medium-term trend indicator,590.6,Price above 50MA is bullish medium-term
3,SPY 200-Day MA,200-day simple moving average - long-term trend indicator,550.13,Price above 200MA is bullish long-term
4,SPY VWAP,Volume Weighted Average Price - shows average price based on volume,526.57,Price above VWAP indicates buying pressure
5,Price vs 20MA (%),Percentage difference between current price and 20-day MA,0.43,Positive % indicates price strength vs 20MA
6,Price vs 50MA (%),Percentage difference between current price and 50-day MA,2.32,Positive % indicates price strength vs 50MA
7,Price vs 200MA (%),Percentage difference between current price and 200-day MA,9.84,Positive % indicates price strength vs 200MA
8,Price vs VWAP (%),Percentage difference between current price and VWAP,14.76,Positive % indicates current buying pressure
9,SPY RSI,Relative Strength Index (14-day) - momentum indicator (0-100),55.91,"RSI > 70 overbought, < 30 oversold"



=== Sector Analysis ===


,Sector,ETF,Return (%),Relative Strength,Momentum (ROC),Volume Trend,Outperforming SPY,Momentum Direction,Relative Strength Rank,Return Rank,Momentum Rank,Overall Score
0,Consumer_Discretionary,XLY,11.07,8.70,6.86,Increasing,Yes,Positive,1,1.0,1.0,9.00
1,Communication Services,XLC,3.98,1.61,0.56,Increasing,Yes,Positive,2,2.0,3.0,7.67
2,Technology,XLK,3.74,1.37,1.34,Increasing,Yes,Positive,3,3.0,2.0,7.33
3,Consumer_Staples,XLP,1.04,-1.33,-1.48,Increasing,No,Negative,4,4.0,4.0,6.00
4,Financials,XLF,-1.19,-3.56,-2.68,Increasing,No,Negative,5,5.0,5.0,5.00
5,Healthcare,XLV,-1.42,-3.79,-5.18,Increasing,No,Negative,6,6.0,9.0,3.00
6,Industrials,XLI,-1.81,-4.18,-4.04,Increasing,No,Negative,7,7.0,7.0,3.00
7,Real_Estate,XLRE,-2.57,-4.94,-3.72,Increasing,No,Negative,8,8.0,6.0,2.67
8,Materials,XLB,-4.08,-6.45,-6.85,Decreasing,No,Negative,9,9.0,10.0,0.67
9,Utilities,XLU,-4.66,-7.03,-4.66,Decreasing,No,Negative,10,10.0,8.0,0.67



=== Custom Filter DF ===


,Sector,ETF,Return (%),Relative Strength,Momentum (ROC),Volume Trend,Outperforming SPY,Momentum Direction,Relative Strength Rank,Return Rank,Momentum Rank,Overall Score
0,Consumer_Discretionary,XLY,11.07,8.70,6.86,Increasing,Yes,Positive,1,1.0,1.0,9.00
1,Communication Services,XLC,3.98,1.61,0.56,Increasing,Yes,Positive,2,2.0,3.0,7.67
2,Technology,XLK,3.74,1.37,1.34,Increasing,Yes,Positive,3,3.0,2.0,7.33



=== Custom Filter Sector List ===


(['Consumer_Discretionary', 'Communication Services', 'Technology'],
 ['XLY', 'XLC', 'XLK'])

True

# 3. Scanner

In [6]:
import scanner
import schedule
from datetime import datetime

market = schedule.MarketSchedule(scan_time='09:00', sleep_time=600) 
sbscan = scanner.StockbotScanner(ib)

# if market.wait_for_scan_time():
#     sbscan.multiscan(scan_code='TOP_PERC_GAIN', price=(1, 100), volume=100_000, change_perc=4, market_cap=100)


# Scan settings so it can be logged and sent via email.
scan_settings = {
    "timestamp": datetime.now().strftime("%Y-%m-%d %H:%M:%S"),
    "scan_code": "TOP_PERC_GAIN",
    "price_min": 1,
    "price_max": 100,
    "volume": 100_000,  # Will be formatted as "100K"
    "market_cap": 100_000_000,  # Will be formatted as "100M"
    "change_percent": 4,
    "total_scanned": 0
}

# if market.wait_for_scan_time():
sbscan.multiscan(
    scan_code=scan_settings['scan_code'], 
    price=(scan_settings['price_min'], scan_settings['price_max']),
    volume=scan_settings['volume'], 
    change_perc=4, 
    market_cap=scan_settings['market_cap']/1_000_000,
    limit_each_cap=20
)

sbscan.scan_results_df

scan_settings['total_scanned'] = len(sbscan.scan_results_df)
scan_settings

# # ------------------- EMAIL -------------------
# from emails import email_summery, email_client

# status_dict['stocks_scanned'] = scan_settings['total_scanned']

# # Create the email generator and generate email
# email_generator = email_summery.StockSummaryEmail()
# email_html = email_generator.generate_email(
#     status_dict,
#     scan_settings= scan_settings,
#     activity_df = None,
#     sector_etf_df = sector_df.round(2),
#     fundamentals_df = None,
#     ta_df = None,
#     daily_ta_df = None
# )

# email_client.send_outlook_email(
#     subject="STOCKBOT: Daily Stock Summery",
#     body=email_html,
#     image_paths=[],
#     recipients=['pary888@gmail.com'],
#     is_html=True)

Scanning STK.US.MAJOR for TOP_PERC_GAIN ..
price: (1, 100), volume: 100000, changePerc: 4, marketCap: (100.0, 1000.0)
STK.US.MAJOR .. len(data): 50
Scanning STK.US.MAJOR for TOP_PERC_GAIN ..
price: (1, 100), volume: 100000, changePerc: 4, marketCap: (1000.0, 10000.0)
STK.US.MAJOR .. len(data): 42
Scanning STK.US.MAJOR for TOP_PERC_GAIN ..
price: (1, 100), volume: 100000, changePerc: 4, marketCap: (10000.0, 100000.0)
STK.US.MAJOR .. len(data): 5


Error 162, reqId 105: Historical Market Data Service error message:API scanner subscription cancelled: 105
Error 162, reqId 106: Historical Market Data Service error message:API scanner subscription cancelled: 106
Error 162, reqId 107: Historical Market Data Service error message:API scanner subscription cancelled: 107
Error 430, reqId 121: The fundamentals data for the security specified is not available.failed to fetch, contract: Stock(symbol='FOX', exchange='SMART', currency='USD')
Error 430, reqId 128: The fundamentals data for the security specified is not available.failed to fetch, contract: Stock(symbol='MSTZ', exchange='SMART', currency='USD')
Error 430, reqId 131: The fundamentals data for the security specified is not available.failed to fetch, contract: Stock(symbol='FOX', exchange='SMART', currency='USD')


{'timestamp': '2024-12-17 21:59:54',
 'scan_code': 'TOP_PERC_GAIN',
 'price_min': 1,
 'price_max': 100,
 'volume': 100000,
 'market_cap': 100000000,
 'change_percent': 4,
 'total_scanned': 45}

In [7]:
sbscan.ta_results_df
scan_settings
sbscan.fund_reults_df
sbscan.scan_results_df

,Rank,Market Cap Range,Symbol,Exchange,Currency,Primary Exchange
0,0,"(100.0, 1000.0)",OPTX,SMART,USD,
40,0,"(10000.0, 100000.0)",TEVA,SMART,USD,
20,0,"(1000.0, 10000.0)",QUBT,SMART,USD,
1,1,"(100.0, 1000.0)",AHG,SMART,USD,
41,1,"(10000.0, 100000.0)",GME,SMART,USD,
21,1,"(1000.0, 10000.0)",HSAI,SMART,USD,
42,2,"(10000.0, 100000.0)",FOXA,SMART,USD,
22,2,"(1000.0, 10000.0)",RGTI,SMART,USD,
2,2,"(100.0, 1000.0)",RZLV,SMART,USD,
3,3,"(100.0, 1000.0)",EHTH,SMART,USD,


# 4. Daily Stock Validation (Fundamentals & TA)

In [8]:
import stock
from strategies import ta
import pandas as pd
import scanner
import schedule




# getting the stock data
# allowed_etfs = ['XLK', 'XLY', 'XLI']

# scan_results_df = pd.DataFrame({
#     'rank': [1, 2, 3],
#     'symbol': ['TSLA', 'AAPL', 'MSFT'],
# })



# permisaable ETFs
allowed_etfs = custom_filter_list[1]
# allowed_etfs = ['XLY', 'XLC', 'XLK']
print(f"Allowed ETFs: {allowed_etfs}")



# see how the scans are validatd here: C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\stock.py --> get_funadmentals_validation_results(), get_TA_validation_results()
sbscan.update_scan_results(allowed_etfs, ib, limit=100)


status_dict['stocks_validated'] = len(sbscan.fund_results_df['Fundamentals Passed']) + len(sbscan.ta_results_df['TA Passed'])

status_dict = {
    "sector_analysis": len(sector_df),
    "stocks_scanned": len(sbscan.scan_results_df),
    "stocks_validated": len(sbscan.fund_results_df['Fundamentals Passed']) + len(sbscan.ta_results_df['TA Passed']),
    "stocks_scored": 0
}

# ------------------- EMAIL -------------------
from emails import email_summery, email_client

# Create the email generator and generate email
email_generator = email_summery.StockSummaryEmail()
email_html = email_generator.generate_email(
    status_dict,
    scan_settings   = scan_settings,
    sector_etf_df   = sector_df.round(2),
    fundamentals_df = sbscan.fund_results_df,
    ta_df           = sbscan.ta_results_df,
    daily_ta_df = None,
    activity_df = None,
)

email_client.send_outlook_email(
    subject="STOCKBOT: Daily Stock Summery",
    body=email_html,
    image_paths=[],
    recipients=['pary888@gmail.com'],
    is_html=True)

Allowed ETFs: ['XLY', 'XLC', 'XLK']
Using cached data for OPTX (age: 2024-12-17 18:04:28)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\OPTX_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\OPTX_1_day.csv
Using cached data for TEVA (age: 2024-12-17 18:04:32)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TEVA_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\TEVA_1_day.csv
Using cached data for QUBT (age: 2024-12-17 18:04:44)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\QUBT_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\QUBT_1_day.csv
Using cached data for AHG (age: 2024-12-17 18:05:11)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\AHG_1_day.csv
Loading data from C:\U

Error 162, reqId 15: Historical Market Data Service error message:Unknown contract, contract: Stock(symbol='MSTZ', exchange='SMART', currency='USD')


Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\MSTZ_1_day.csv
Using cached data for CANG (age: 2024-12-17 19:54:41)
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\CANG_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\CANG_1_day.csv


True

In [19]:
sbscan.fund_results_df.sort_values('Fundamentals Passed', ascending=False)

,Rank,Market Cap Range,Symbol,Exchange,Currency,Primary Exchange,Sector1 Valid,Sector2 Valid,Market Cap > 300M,Vol 10DayMA > 300K,Fundamentals Passed
7,7,"(100.0, 1000.0)",CANG,SMART,USD,,True,True,True,True,True
43,3,"(10000.0, 100000.0)",TME,SMART,USD,,True,True,True,True,True
20,0,"(1000.0, 10000.0)",QUBT,SMART,USD,,True,True,True,True,True
41,1,"(10000.0, 100000.0)",GME,SMART,USD,,True,True,True,True,True
42,2,"(10000.0, 100000.0)",FOXA,SMART,USD,,True,True,True,True,True
24,4,"(1000.0, 10000.0)",QBTS,SMART,USD,,True,True,True,True,True
6,6,"(100.0, 1000.0)",FTEL,SMART,USD,,True,False,True,False,False
5,5,"(100.0, 1000.0)",SKYT,SMART,USD,,False,True,True,True,False
25,5,"(1000.0, 10000.0)",TDW,SMART,USD,,False,False,True,True,False
4,4,"(100.0, 1000.0)",CRDF,SMART,USD,,False,False,False,True,False


In [18]:
cols = [c for c in sbscan.ta_results_df.columns if c not in ['Exchange', 'Currency', 'Primary Exchange']]

sbscan.ta_results_df.sort_values('TA Passed', ascending=False)[cols]
sbscan.scan_results_df.sort_values('TA Passed', ascending=False)[cols]

,Rank,Market Cap Range,Symbol,Close > $1,Above 200MA,Above 150MA,Breaks Above 50MA,50MA Slope > 0,Gap Up > 4%,Volume > 50K,Volume Above 10MA,Volume Dev > 80%,TA Passed
40,0,"(10000.0, 100000.0)",TEVA,True,True,True,True,True,True,True,True,True,True
0,0,"(100.0, 1000.0)",OPTX,True,True,True,False,True,True,True,False,True,False
26,6,"(1000.0, 10000.0)",MSTZ,True,False,False,False,False,True,True,False,False,False
6,6,"(100.0, 1000.0)",FTEL,True,True,True,False,True,True,False,False,False,False
5,5,"(100.0, 1000.0)",SKYT,True,True,True,False,True,True,False,False,True,False
25,5,"(1000.0, 10000.0)",TDW,True,False,False,False,False,True,False,True,False,False
24,4,"(1000.0, 10000.0)",QBTS,True,True,True,False,True,True,True,False,False,False
4,4,"(100.0, 1000.0)",CRDF,True,True,True,False,True,True,False,False,False,False
44,4,"(10000.0, 100000.0)",FOX,True,True,True,False,True,False,False,False,False,False
43,3,"(10000.0, 100000.0)",TME,True,False,False,True,False,True,True,False,False,False


In [10]:
def get_stock_fundamentals(ib, ticker ):
    """Retrieve comprehensive stock information including all available ratios."""
    contract = Stock(ticker, 'SMART', 'USD')
    details = ib.reqContractDetails(contract)
    if not details:
        raise ValueError(f"No contract details found for {ticker}")
    
    fundamental_data = ib.reqFundamentalData(contract, 'ReportSnapshot')
    return fundamental_data

not get_stock_fundamentals(ib, 'FOX')

True

# 5. Daily Stock Scores (TA)

In [11]:
import os

# Go up one level from scanner folder, then into data folder
current_dir = os.getcwd()  # Gets current directory (scanner)
parent_dir = os.path.dirname(current_dir)  # Goes up one level to stockbot4
data_folder = os.path.join(parent_dir, 'data')  # Goes into data folder
os.chdir(data_folder)

import historical_data as hd

symbol = 'ROKU'
interval = '1 day'
# data = hd.get_hist_data(symbol, '52 weeksAgo', 'now', interval)
data = hd.get_hist_data(symbol, '2022-12-30', '2024-11-14', interval)
display(data)

Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\ROKU_1_day.csv
Loading data from C:\Users\sidsu\anaconda3\envs\SB4\stockbot4\data\historical_data_store\ROKU_1_day.csv


,open,high,low,close,volume
date,,,,,
2022-12-30,40.30,40.97,39.05,40.75,42515
2023-01-03,41.45,42.37,39.00,40.51,51584
2023-01-04,40.93,43.28,39.84,42.47,68893
2023-01-05,42.59,44.25,40.93,42.81,78793
2023-01-06,42.75,44.46,41.30,43.60,56965
2023-01-09,44.04,46.49,43.80,45.09,54379
2023-01-10,45.14,47.32,44.18,46.95,38954
2023-01-11,47.17,49.00,46.80,48.92,42734
2023-01-12,47.10,49.41,46.31,49.15,69237


# 6. Daily TA

# 7. Filter TA

In [12]:
# BreaksMA = 200 // yesterday below,  today above
# BreaksMA = 50 // yesterday below,  today above
# > MA200
# BreaksVolumeMA = 10 // breaks 10 day vol ma
# MA50isRisingNthDays = (3, 10) // viable range midpoint is best score
# With Segment trend
# With Mkt trend 


ta_1 = ta.Breaks('close', 'MA_cl_200', True) # BreaksMA = 200
ta_2 = ta.Breaks('close', 'MA_cl_50', True) # BreaksMA = 50
ta_3 = ta.AboveBelow('close', 'MA_cl_50', True) # > MA50
ta_4 = ta.AboveBelow('close', 'MA_cl_200', True) # > MA200
ta_5 = ta.AboveBelow(3, 'TDUR_MA_cl_50', True) # MA50isRisingNthDays
ta_6 = ta.AboveBelow(80, 'VDEV_10', True) # > 80% above MA10

f.add_ta(ta_1)
f.add_ta(ta_2)
f.add_ta(ta_3)
f.add_ta(ta_4)
f.add_ta(ta_5)
f.add_ta(ta_6)

ta_filter_names = [ta_1.name, ta_2.name, ta_3.name, ta_4.name, ta_5.name, ta_6.name]

df = f.data.copy()

# add column to sum up the scores
df['filter_score'] = df[ta_filter_names].sum(axis=1)
df

# filter out the rows with score less than 3
filtered_df = df[df['filter_score'] >= 5]
filtered_df

f.load_ohlcv(data)
f.update_ta_data()
f.data

NameError: name 'f' is not defined

# 8. Daily Scores

In [ ]:
df = f.data.copy()

# add column to sum up the scores
df['filter_score'] = df[ta_filter_names].sum(axis=1)
df

# filter out the rows with score less than 3
filtered_df = df[df['filter_score'] >= 5]
filtered_df

,open,high,low,close,volume,MA_cl_200,MA_cl_50,MA_vo_10,ACC_close,VDEV_10,TDUR_MA_cl_50,BRK_UP_MA,ABV_close_MA_cl_50,ABV_close_MA_cl_200,ABV_3_TDUR_MA_cl_50,ABV_80_VDEV_10,filter_score,all_true
date,,,,,,,,,,,,,,,,,,
2023-06-01,58.13,59.27,56.48,58.88,30579,NaN,58.8208,42705.7,7.60,-28.395975,-25,True,True,False,True,True,5,NaN
2023-11-02,70.27,79.12,69.00,77.57,332103,66.79200,71.4652,77477.6,24.07,328.643892,-21,True,True,True,True,False,5,NaN
2024-01-08,88.90,94.90,88.85,93.56,55274,74.14190,90.6266,42749.5,-18.88,29.297419,39,True,True,True,False,True,5,NaN
2024-02-07,92.89,96.06,91.87,94.75,39558,76.91170,94.0274,34138.4,7.47,15.875378,-7,True,True,True,True,True,6,NaN
2024-05-17,61.50,61.86,60.60,61.45,14528,76.73030,61.3974,27163.7,9.47,-46.516859,-77,True,True,False,True,True,5,NaN
2024-06-27,57.16,59.51,56.77,59.20,33846,73.33430,57.9254,26429.2,10.81,28.062900,1,True,True,False,True,True,5,NaN
2024-07-26,58.28,59.28,57.42,59.28,21820,72.27505,58.8060,28224.2,-22.13,-22.690457,-3,True,True,False,True,True,5,NaN
2024-07-31,57.30,59.11,56.94,58.70,20492,72.08275,58.5948,25091.4,-13.28,-18.330583,-6,True,True,False,True,True,5,NaN
2024-08-15,59.45,61.00,55.88,59.30,31276,71.69800,58.0256,31683.0,15.23,-1.284601,1,True,True,False,True,True,5,NaN


# 9. Final Watch List